In [43]:
import pandas as pd
import datetime as dt

In [37]:
eleves = pd.read_csv('../data/DIM_ELEVE.csv', parse_dates=['DATE_NAISSANCE'])

In [38]:
print(eleves.columns)
print(eleves.shape)
print(eleves.GENRE.unique())

Index(['ID_ELEVE', 'GENRE', 'DATE_NAISSANCE', 'ID_CLASSE'], dtype='object')
(10000, 4)
['m' 'f']


In [34]:
dist_genre = eleves.groupby('GENRE').count()['ID_ELEVE']

In [56]:
reponses = pd.read_csv('../data/FACT_REPONSE.csv', parse_dates=['DATE_TIME'])

In [57]:
print(reponses.columns)
print(reponses.shape)

Index(['ID_LOG', 'DATE_TIME', 'KEY_REPONSE', 'ID_REPONDANT'], dtype='object')
(100000, 4)


In [61]:
reponses.columns

Index(['ID_LOG', 'DATE_TIME', 'KEY_REPONSE', 'ID_REPONDANT'], dtype='object')

In [62]:
merged = reponses.merge(
    eleves, left_on = "ID_REPONDANT", right_on = "ID_ELEVE"
    )

In [76]:
merged['age_group'] = pd.cut(
    # delta t annees
    (merged.DATE_TIME - merged.DATE_NAISSANCE).dt.days // 365.2425,
    bins = [0, 11, 13, 15, 17, 99],
    labels = ['0-11', '11-12', '13-14', '15-17', '>17'],
    right = False
)

In [77]:
merged.groupby("age_group").count()

/tmp/ipykernel_1528263/2799898231.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged.groupby("age_group").count()


,ID_LOG,DATE_TIME,KEY_REPONSE,ID_REPONDANT,ID_ELEVE,GENRE,DATE_NAISSANCE,ID_CLASSE
age_group,,,,,,,,
0-11,10788,10788,10788,10788,10788,10788,10788,10788
11-12,15713,15713,15713,15713,15713,15713,15713,15713
13-14,17369,17369,17369,17369,17369,17369,17369,17369
15-17,17598,17598,17598,17598,17598,17598,17598,17598
>17,38532,38532,38532,38532,38532,38532,38532,38532


In [27]:
n_eleves = eleves.shape[0]

In [28]:
responses.KEY_REPONSE.value_counts()['B_2_1_d'] / n_eleves

0.0605

In [35]:
merged.loc[merged.KEY_REPONSE == 'B_2_1_d'].groupby("GENRE").count()['ID_LOG'] / dist_genre

GENRE
f    0.055634
m    0.065326
dtype: float64